In [1]:
import os
from neo4j import GraphDatabase

# 从环境变量加载Neo4j连接信息
NEO4J_URI = os.getenv("NEO4J_URI", "bolt://localhost:7687")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME", "neo4j")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD", "neo4jneo4j")

class Neo4jTree:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def create_tree(self):
        with self.driver.session() as session:
            # 创建节点和关系
            session.write_transaction(self._create_tree)

    @staticmethod
    def _create_tree(tx):
        query = """
        CREATE (a:Node {name: 'A'})
        CREATE (b:Node {name: 'B'})
        CREATE (c:Node {name: 'C'})
        CREATE (d:Node {name: 'D'})
        CREATE (e:Node {name: 'E'})
        CREATE (f:Node {name: 'F'})
        CREATE (g:Node {name: 'G'})
        CREATE (a)-[:CHILD]->(b)
        CREATE (a)-[:CHILD]->(c)
        CREATE (b)-[:CHILD]->(d)
        CREATE (b)-[:CHILD]->(e)
        CREATE (c)-[:CHILD]->(f)
        CREATE (c)-[:CHILD]->(g)
        """
        tx.run(query)

# 连接并创建树
neo4j_tree = Neo4jTree(NEO4J_URI, NEO4J_USERNAME, NEO4J_PASSWORD)
neo4j_tree.create_tree()
neo4j_tree.close()

/tmp/ipykernel_964125/2935929723.py:19: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._create_tree)


In [2]:
from langchain.chains import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph
# from langchain_openai import ChatOpenAI

graph = Neo4jGraph(url="bolt://localhost:7687", username="neo4j", password="neo4jneo4j")


In [3]:
graph.query(
    """
MERGE (m:Movie {name:"Top Gun", runtime: 120})
WITH m
UNWIND ["Tom Cruise", "Val Kilmer", "Anthony Edwards", "Meg Ryan"] AS actor
MERGE (a:Actor {name:actor})
MERGE (a)-[:ACTED_IN]->(m)
"""
)

[]

In [4]:
print(graph.schema)

Node properties:

Relationship properties:

The relationships:



In [32]:
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(model="qwen:72b", temperature=0.8)
# llm = ChatOllama(model="llama:70b", temperature=0.8)
# chain = GraphCypherQAChain.from_llm(
#     llm, graph=graph, verbose=True
# )

In [37]:
from langchain_core.prompts.prompt import PromptTemplate

CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
Examples: Here are a few examples of generated Cypher statements for particular questions:
# How many people played in Top Gun?
MATCH (m:Movie {{title:"Top Gun"}})<-[:ACTED_IN]-()
RETURN count(*) AS numberOfActors

The question is:
{question}"""

CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], template=CYPHER_GENERATION_TEMPLATE
)

chain = GraphCypherQAChain.from_llm(
    llm,
    graph=graph,
    verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
    validate_cypher=True,
)

In [39]:
chain.invoke({"query": "Who played in Top Gun?"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {name:"Top Gun"})<-[:ACTED_IN]-(a:Actor)
RETURN a.name AS actorName
Full Context:
[{'actorName': 'Tom Cruise'}, {'actorName': 'Val Kilmer'}, {'actorName': 'Anthony Edwards'}, {'actorName': 'Meg Ryan'}]

> Finished chain.


{'query': 'Who played in Top Gun?',
 'result': 'Top Gun featured Tom Cruise, Val Kilmer, Anthony Edwards, and Meg Ryan in its cast.'}